<a href="https://colab.research.google.com/github/Oukey/M_L/blob/master/myHW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Занятие 6. Выполнение задания

Импорт нужных пакетов

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

Обработка стандартного датасета **CIFAR10**(фотографии десяти классов: автомобиль, самолет, птица, кот, олень, собака, лягушка, лошадь, корабль и грузовик). 60000 цветных изображений (50000 обучающих и 10000 тестовых)  размером 32*32 пиксела (RGB) 

In [0]:
input_size = 3 * 32 * 32  # Количество цветов * размер изображения в точках
num_classes = 10          # Количество классов (10 видов изображений)
n_epochs = 2              # Количество эпох
batch_size = 4            # Размер мини-пакета входных данных
lr = 0.001                # Скорость обучения 

Выполнение композиции трансформации с выполнением преобразования в тензоры с последующей нормализацией (преобразование цветных изображений в серую палитру).

In [7]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Обучающий датасет
cifar_trainset = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# Тестовый датасет
cifar_testset = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)

print('Размер обучающего датасета:', len(cifar_trainset))
print('Размер тестового датасета:', len(cifar_testset))

Files already downloaded and verified
Files already downloaded and verified
Размер обучающего датасета: 50000
Размер тестового датасета: 10000


Загрузка данных для обучения и тестирования

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=cifar_testset, batch_size=batch_size, shuffle=True)

Инициализация девайса и стандартная фукция "шаг обучения"

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

Модель обучения